In [16]:
consumer_key='----------------------'
consumer_secret='-------------------'
access_token ='---------------------'
access_secret='---------------------'

In [28]:
from tweepy import API 
from tweepy import Cursor
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import tweepy
import numpy as np
import pandas as pd


# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


# # # # TWITTER AUTHENTICATER # # # #
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        return auth

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
    """
    Class for streaming and processing live tweets.
    """
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)


# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
    """
    This is a basic listener that just prints received tweets to stdout.
    """
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)


class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """
    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])


        return df

 
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = tweepy.Cursor(api.search,q='Lionel Messi',lang="en",tweetmode='extended').items(1000)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    #df.to_csv(r'C:\Users\ahmed hatem\Downloads\live twiiter data\live tweets for testing.csv')
    print(df.head(100))

                                               Tweets
0   RT @SkySportsNews: Mauricio Pochettino has def...
1   RT @MrFilipeOrlando: Put Cristiano Ronaldo in ...
2   RT @goal: Cristiano Ronaldo since joining Man ...
3   RT @Elite_CR7: Lionel Messi has gone past 112 ...
4   RT @CrewsMat10: Lionel Messi received a blow a...
..                                                ...
95  RT @goal: Cristiano Ronaldo since joining Man ...
96  RT @cr7raprhymes: Cristiano Ronaldo scored two...
97  RT @Elite_CR7: Lionel Messi has gone past 112 ...
98  RT @goal: Lionel Messi makes his first appeara...
99  RT @goal: Cristiano Ronaldo since joining Man ...

[100 rows x 1 columns]


# this is an attempt to follow the steps in this link : 
# however i had an error for two days and couldn't find a solution online

In [19]:
import tweepy
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
import socket
import json

In [20]:
class TweetsListener(StreamListener):
  # tweet object listens for the tweets
  def _init_(self, csocket):
    self.client_socket = csocket
  def on_data(self, data):
    try:  
      msg = json.loads( data )
      print("new message")
      # if tweet is longer than 140 characters
      if "extended_tweet" in msg:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['extended_tweet']['full_text']+"t_end")\
            .encode('utf-8'))         
        print(msg['extended_tweet']['full_text'])
      else:
        # add at the end of each tweet "t_end" 
        self.client_socket\
            .send(str(msg['text']+"t_end")\
            .encode('utf-8'))
        print(msg['text'])
      return True
    except BaseException as e:
        print("Error on_data: %s" % str(e))
    return True
  def on_error(self, status):
    print(status)
    return True

In [21]:
def sendData(c_socket, keyword):
  print('start sending data from Twitter to socket')
  # authentication based on the credentials
  auth = OAuthHandler(consumer_key, consumer_secret)
  auth.set_access_token(access_token, access_secret)
  # start sending data from the Streaming API 
  twitter_stream = Stream(auth, TweetsListener(c_socket))
  twitter_stream.filter(track = keyword, languages=["en"])

In [22]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import functions as F
from textblob import TextBlob

In [23]:
def preprocessing(lines):
    words = lines.select(explode(split(lines.value, "t_end")).alias("word"))
    words = words.na.replace('', None)
    words = words.na.drop()
    words = words.withColumn('word', F.regexp_replace('word', r'http\S+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '@\w+', ''))
    words = words.withColumn('word', F.regexp_replace('word', '#', ''))
    words = words.withColumn('word', F.regexp_replace('word', 'RT', ''))
    words = words.withColumn('word', F.regexp_replace('word', ':', ''))
    return words

In [24]:
def polarity_detection(text):
    return TextBlob(text).sentiment.polarity
def subjectivity_detection(text):
    return TextBlob(text).sentiment.subjectivity
def text_classification(words):
    # polarity detection
    polarity_detection_udf = udf(polarity_detection, StringType())
    words = words.withColumn("polarity", polarity_detection_udf("word"))
    # subjectivity detection
    subjectivity_detection_udf = udf(subjectivity_detection, StringType())
    words = words.withColumn("subjectivity", subjectivity_detection_udf("word"))
    return words

In [25]:

if __name__ == '__main__':
    spark = SparkSession.builder.appName("TwitterSentimentAnalysis").getOrCreate()
    # read the tweet data from socket
    print('session started')
    lines = spark.readStream.format("socket").option("host", "0.0.0.0").option("port", 5555).load()
    # Preprocess the data
    words = preprocessing(lines)
    # text classification to define polarity and subjectivity
    words = text_classification(words)
    words = words.repartition(1)
    query = words.writeStream.queryName("Ahmed").option("host", "localhost")\
        .outputMode("append").format("csv")\
        .option("path", "./parc")\
        .option("checkpointLocation", r"C:\Users\ahmed hatem\Downloads\try")\
        .trigger(processingTime='60 seconds').start()
    query.awaitTermination()


session started


StreamingQueryException: Connection refused: connect
=== Streaming Query ===
Identifier: Ahmed [id = a691bb33-6c7d-4852-9ca3-432f9a19cc35, runId = 71c2f108-d803-4b9b-9c7d-90171c9d25bc]
Current Committed Offsets: {}
Current Available Offsets: {TextSocketV2[host: 0.0.0.0, port: 5555]: -1}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
Repartition 1, true
+- Project [word#17, polarity#20, subjectivity_detection(word#17) AS subjectivity#24]
   +- Project [word#17, polarity_detection(word#17) AS polarity#20]
      +- Project [regexp_replace(word#15, :, , 1) AS word#17]
         +- Project [regexp_replace(word#13, RT, , 1) AS word#15]
            +- Project [regexp_replace(word#11, #, , 1) AS word#13]
               +- Project [regexp_replace(word#9, @\w+, , 1) AS word#11]
                  +- Project [regexp_replace(word#6, http\S+, , 1) AS word#9]
                     +- Filter AtLeastNNulls(n, word#6)
                        +- Project [CASE WHEN (word#3 = ) THEN cast(null a